In [1]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock'}`


Cloudflare Stock: Strong Key Metrics Driving The Rise In Share Price. Cloudflare is delivering strong revenue growth, driven by enterprise customers and large contracts, especially for its Workers developer platform. The company's revamped sales strategy, focusing on la... Real-time Cloudflare stock price (NYSE: NET) plus other valuable data points like day range, year, stock analyst insights, related news and more. Get Cloudflare Inc (NET) real-time stock quotes, news, price and financial information from Reuters to inform your trading and investments ... Cloudflare Inc NET. Official Data Partner. Latest ... Cloudflare's stock has recently been incredibly volatile, ranging between approximately $30 and $80. The significant price movements have been in response to news and events affecting the broader tech industry. Cloudflare operates in the highly competitive cybers

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "Cloudflare's stock symbol is NET and it is listed on the New York Stock Exchange (NYSE). Cloudflare is a connectivity cloud company that offers website and application services for security, performance, and reliability.\n\nCloudflare's stock has been experiencing significant volatility, with price movements ranging between approximately $30 and $80. This volatility is influenced by news and events affecting the broader tech industry.\n\nTo analyze if Cloudflare is a potential good investment, we need to consider various factors such as the company's financial performance, growth prospects, competitive landscape, and industry trends. It's important to conduct a thorough analysis and consider consulting with a financial advisor before making any investment decisions."}